In [1]:
import nnutil 
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import torch
from torch import nn
import math
device = nnutil.device
device

device = cuda


device(type='cuda')

In [2]:
import pandas as pd
df = pd.read_csv('TCGA-BRCA-Integrated-Data-Sampler.csv')


In [4]:
# 加载数据
dataloader, data_tensor, normalized_tensor,data_y,ae_mean, ae_std = nnutil.data_prehandle(
    file='TCGA-BRCA-Integrated-Data.csv', 
    df = df,
    start=0,
    label='label'
)

data.shape:(2110, 2048), data_tensor:torch.Size([2110, 2048])


In [5]:
data_y.value_counts()

4    422
0    422
2    422
3    422
1    422
Name: label, dtype: int64

In [6]:
nnutil.setup_seed()

In [7]:
normalized_tensor

tensor([[ 0.9438, -1.1304, -1.1308,  ..., -1.1335, -0.1454,  1.0683],
        [-1.1374, -1.1179, -1.0755,  ..., -1.1367, -0.8094, -1.1134],
        [-0.2025, -1.1326, -1.1346,  ..., -1.1288,  0.9545,  1.0807],
        ...,
        [-1.0826, -1.1372, -1.1366,  ..., -1.1373, -0.0879,  0.3378],
        [-0.7217, -1.1369, -1.1331,  ..., -1.1311, -0.1615,  0.0237],
        [-0.9587, -1.1363, -1.1344,  ..., -1.1319, -0.0351,  0.9200]])

In [8]:
# 将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(normalized_tensor, data_y, test_size=0.2, random_state=42)
X_train.shape

torch.Size([1688, 2048])

In [9]:
# 转化为 PyTorch 张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)
X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor

/tmp/ipykernel_445425/1342047294.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
/tmp/ipykernel_445425/1342047294.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test_tensor = torch.tensor(X_test, dtype=torch.float32)


(tensor([[-1.1374, -1.1180, -1.1229,  ..., -1.1174,  0.0290,  1.0825],
         [ 0.1905, -1.1362, -1.1188,  ..., -1.1374, -1.1274,  0.4164],
         [ 1.0611, -1.1371, -1.1371,  ..., -1.0889, -0.0795,  0.1145],
         ...,
         [-0.9189, -1.1363, -1.1343,  ..., -1.1229, -0.5819,  1.0731],
         [-0.2807, -1.1320, -1.1223,  ..., -1.0424,  0.6984,  1.0564],
         [-0.7677, -1.0779, -0.9763,  ..., -0.0561,  0.9293,  1.0791]]),
 tensor([1, 2, 4,  ..., 1, 1, 0]),
 tensor([[-0.4620, -0.4157, -1.1293,  ..., -1.1340,  1.0860,  1.0863],
         [-0.9752, -1.1328, -1.1251,  ..., -1.1208,  0.8190,  1.0437],
         [-0.4788, -1.1367, -1.1369,  ..., -1.1369,  0.9599,  0.9342],
         ...,
         [-1.1060, -1.1291, -1.1270,  ..., -1.1291,  0.2477,  0.8012],
         [ 0.9956, -1.1246, -1.1303,  ..., -1.1372,  0.8230,  1.0863],
         [-0.9925, -1.1218, -1.1362,  ..., -1.1304,  0.6845,  1.0343]]),
 tensor([2, 4, 2, 2, 0, 2, 4, 2, 3, 0, 3, 4, 3, 3, 4, 0, 4, 3, 1, 0, 3, 0, 4, 1,


In [10]:
nnutil.count_num(y_train_tensor)

v: 0 ,num: 343
v: 1 ,num: 346
v: 2 ,num: 325
v: 3 ,num: 331
v: 4 ,num: 343


In [11]:
nnutil.count_num(y_test_tensor)

v: 0 ,num: 79
v: 1 ,num: 76
v: 2 ,num: 97
v: 3 ,num: 91
v: 4 ,num: 79


In [12]:
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, dim_feedforward=2048, nhead=8, nlayers=6, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.input_dim = input_dim
        self.src_mask = None
        self.encoder_num = 1024
        self.fc = nn.Linear(self.input_dim, self.encoder_num)
        self.softmax = nn.Softmax(dim=1)
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.encoder_num,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=nlayers)
        self.classifier = nn.Linear(self.encoder_num, num_classes)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            mask = self._generate_square_subsequent_mask(len(src)).to(src.device)
            self.src_mask = mask
        src = self.fc(src)
        src = src * math.sqrt(self.encoder_num)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.classifier(output)
        return torch.squeeze(output)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [13]:

train_dataset = nnutil.CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = nnutil.CustomDataset(X_test_tensor, y_test_tensor)
batch_size = 128
input_dim = X_train_tensor.shape[1]
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)
trans = TransformerClassifier(input_dim=input_dim, num_classes=5).to(device)
para_file_dir = 'model/gen-met'

In [14]:
epoch_losses = []
epoch_test_losses = []

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import math

def train_transformer(num_epochs=500, para_file='best_model.pth', model = None, train_loader = None):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    num_epochs = num_epochs
    min_loss = 10000
    best_file = para_file
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        epoch_test_loss = 0
        num = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            num += 1
            data = data.unsqueeze(1).to(device)  
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            epoch_loss += loss.item()
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
        epoch_loss = epoch_loss / num
        epoch_losses.append(epoch_loss)
        if (epoch_loss < min_loss):
            min_loss = epoch_loss
            print(f'save best model, epoch {epoch} loss = {min_loss}')
            torch.save(model.state_dict(), best_file)
        
        
        # print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss}')
        ### Testing
        model.eval()
        num = 0
        with torch.inference_mode():
            for batch_idx, (data_test, target_test) in enumerate(train_loader):
                num +=1
                # 1. Forward pass
                data_test = data_test.to(device)
                target_test = target_test.to(device)
                test_logits = model(data_test)
                test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
                # 2. Calculate test loss and accuracy
                test_loss = criterion(test_logits, target_test.to(device))
                epoch_test_loss += test_loss.item()
        epoch_test_loss = epoch_test_loss / num
        epoch_test_losses.append(epoch_test_loss)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {epoch_loss}, Test Loss: {epoch_test_loss}')

In [16]:
train_transformer(num_epochs=5000, para_file=f'{para_file_dir}/best_model_gen-met_{input_dim}.pth', model = trans, train_loader = train_loader)

In [17]:
nnutil.test_transformer(para_file=f'{para_file_dir}/best_model_gen-met.pth', model = trans, test_loader = test_loader)

Accuracy of the model on the test set: 91.23222748815166%
AUC: 0.9862324609538667
Recall: 0.9218201347132575
Precision: 0.9197604523361436
F1 score: 0.9159938016973566


(91.23222748815166,
 0.9862324609538667,
 0.9218201347132575,
 0.9197604523361436,
 0.9159938016973566)

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score, classification_report
def test_transformer_subclass(para_file="best_model.pth", model=None, test_loader=None):
    model.load_state_dict(torch.load(para_file))
    model.eval()
    total = 0
    correct = 0
    predicted_probs = []
    targets = []
    
    with torch.no_grad():
        for data, target in test_loader:
            data = data.unsqueeze(1).to(device)
            target = target.to(device)
            output = model(data)
            predicted_probs.extend(
                torch.softmax(output, dim=1).tolist()
            )  
            targets.extend(target.tolist())
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = 100 * correct / total
    print(f"Overall Accuracy of the model on the test set: {accuracy}%")

    auc = roc_auc_score(targets, predicted_probs, multi_class="ovr")
    print(f"Overall AUC: {auc}")

    predicted_labels = np.argmax(predicted_probs, axis=1)

    # Generate classification report for each class
    report = classification_report(targets, predicted_labels, output_dict=True)

    num_classes = len(set(targets))
    targets = np.array(targets)
    label_size = [0 for i in range(num_classes)]
    correct_label_size = [0 for i in range(num_classes)]
    
    for target, pred in zip(targets, predicted_labels):
        label_size[target] += 1
        if target == pred:
            correct_label_size[target] += 1
    print(f'{label_size=}')
    print(f'{correct_label_size=}')
    
    for class_label in range(num_classes):
        # Calculate per-class AUC
        
        class_probs = np.array(predicted_probs)[:, class_label]  # Probabilities for the current class
        class_auc = roc_auc_score((targets == class_label).astype(int), class_probs)
        class_label = str(class_label)
        print(f"*************************")
        print(f"  Class {class_label}:")
        print(f"  Precision: {report[class_label]['precision']:.4f}")
        print(f"  Recall: {report[class_label]['recall']:.4f}")
        print(f"  F1 Score: {report[class_label]['f1-score']:.4f}")
        print(f"  AUC: {class_auc:.4f}")
        print(f"*************************")
        print()
    return accuracy, auc

In [68]:
test_transformer_subclass(para_file=f'{para_file_dir}/best_model_gen-met.pth', model = trans, test_loader = test_loader)

Overall Accuracy of the model on the test set: 91.23222748815166%
Overall AUC: 0.9862324609538667
label_size=[79, 76, 97, 91, 79]
correct_label_size=[77, 76, 69, 84, 79]
*************************
  Class 0:
  Precision: 0.9747
  Recall: 0.9747
  F1 Score: 0.9747
  AUC: 0.9993
*************************

*************************
  Class 1:
  Precision: 0.8837
  Recall: 1.0000
  F1 Score: 0.9383
  AUC: 0.9994
*************************

*************************
  Class 2:
  Precision: 0.9452
  Recall: 0.7113
  F1 Score: 0.8118
  AUC: 0.9565
*************************

*************************
  Class 3:
  Precision: 0.8077
  Recall: 0.9231
  F1 Score: 0.8615
  AUC: 0.9764
*************************

*************************
  Class 4:
  Precision: 0.9875
  Recall: 1.0000
  F1 Score: 0.9937
  AUC: 0.9996
*************************



(91.23222748815166, 0.9862324609538667)

In [ ]:
import matplotlib.pyplot as plt

def plot_losses(epoch_losses):
    plt.figure(figsize=(6,5))
    plt.plot(epoch_losses)
    plt.title("Training Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

In [ ]:
import json
loss_result = {}
shape = X_train_tensor.shape[1]
loss_result["dimension"] = shape
loss_result["train_loss"] = epoch_losses
loss_result["test_loss"] = epoch_test_losses
name = f"loss_dim_{shape}.json"
with open(name, "w") as f:
    json.dump(loss_result, f)

In [ ]:
plot_losses(epoch_losses)

In [ ]:
plot_losses(epoch_test_losses)